### Load data:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

coin_ids = {
    'Bitcoin': 1442,
    'Ripple': 1444,
    'Ethereum': 1443,
    'Bitcoin Cash': 1445,
    'Bitcoin Gold': 1456,
    'Cardano': 1446,
    'Dash': 1453,
    'Dogecoin': 1477,
    'Eos': 1452,
    'Ethereum Classic': 1457,
    'Iota': 1451,
    'Lisk': 1460,
    'Litecoin': 1448,
    'Monero': 1454,
    'NEMcoin': 1447,
    'Neo': 1449,
    'Stellar': 1450,
    'Tether': 1474,
    'Tron': 1455,
    'Zcash': 1465,
}

id_coins = {v: k for k, v in coin_ids.items()}

### load data:

iterator_full_data = pd.read_csv('data/datathon.csv', iterator=True, chunksize=100000, parse_dates=['time'])
subset_full_data = pd.concat([chunk[chunk.refID_coin.isin(coin_ids.values())] for chunk in iterator_full_data])
subset_full_data = subset_full_data.replace({'refID_coin': id_coins})

### Fix missing value issues:

In [2]:
### There are some duplicate rows, as well as some rows that should exist, but don't:
subset_full_data[subset_full_data[['time', 'refID_coin']].duplicated()].shape

first_period = subset_full_data.time.min()
last_period = subset_full_data.time.max()
all_periods = pd.date_range(start=first_period, end=last_period, freq='5min')
all_currencies = subset_full_data.refID_coin.unique()
full_index = pd.MultiIndex.from_product([all_periods, all_currencies])
subset_full_data = (subset_full_data
                    .drop_duplicates(subset=['time', 'refID_coin'])
                    .set_index(['time', 'refID_coin'])
                    .reindex(full_index, fill_value = np.nan)
                    .reset_index()
                    .rename({'level_0': 'time', 'level_1': 'refID_coin'}, axis=1))

### add market cap percentages:

In [3]:
### fill missing values:
numeric_cols = ['price', 'marketCap', 'CirculatingSupply', 'Volume24h', 'Movement1h', 'Movement24h']

subset_full_data[numeric_cols] = subset_full_data.groupby('refID_coin')[numeric_cols].transform(lambda x: x.fillna(method='ffill'))

### Calculate marketCap % for each currency by period

market_caps = subset_full_data[['time', 'refID_coin', 'marketCap']]
market_caps = market_caps.pivot(index='time', columns='refID_coin', values='marketCap')
market_caps['total_market_cap'] = market_caps.sum(axis=1)

for currency in coin_ids.keys():
    market_caps[currency] = market_caps[currency] / market_caps['total_market_cap']

# market_caps.drop('total_market_cap', axis=1, inplace=True) 
market_caps.head()

### save just in case
market_caps.to_csv('data/market_caps_by_period.csv')

### Unpivot market caps, so that I can join them to total_subset_data
I realize that's ugly, but I think it's not possible to be cleaner - see https://github.com/pandas-dev/pandas/issues/17676

In [4]:
unpivoted_market_caps = market_caps.reset_index()\
                        .drop(columns = ['total_market_cap'], axis=1)\
                        .melt(id_vars = ['time'], value_vars=subset_full_data.refID_coin.unique().tolist())\
                        .rename({'value': 'percent_market_cap'}, axis=1)

In [5]:
def calculate_entropy(values):
    values /= sum(values)
    return - sum(values * np.log2(values.astype('float64'))) 
market_caps['market_entropy'] = market_caps.apply(lambda x: calculate_entropy(x[coin_ids.keys()]), axis=1)
market_caps = market_caps.reset_index()[['time', 'market_entropy', 'total_market_cap']]

In [6]:
subset_full_data = pd.merge(subset_full_data, unpivoted_market_caps, how = 'outer', on = ['time', 'refID_coin'])
subset_full_data = pd.merge(subset_full_data, market_caps, how = 'left', on = ['time'])
subset_full_data.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

### Create moving variances:

In [7]:
regex = "marketCap|CirculatingSupply|Volume24h|market_entropy|total_market_cap|price"
log_cols = subset_full_data.filter(regex = regex).columns
subset_full_data.loc[:, log_cols] = np.log(subset_full_data.loc[:, log_cols].values+1)

In [8]:
var_cols = ['total_market_cap']
all_numeric_cols = [] + numeric_cols
for column in var_cols:
    for center_of_mass in [72]:
        new_col_name = column + '_MA_VAR_' + str(center_of_mass)
        all_numeric_cols.append(new_col_name)
        ma = subset_full_data.groupby("refID_coin", as_index = False).apply(lambda x: x[column].ewm(com=center_of_mass).std())
        subset_full_data[new_col_name] = ma.reset_index(level=0, drop=True)

### Create moving averages:

In [ ]:
high_priority_cols = ['price', 'CirculatingSupply', 'percent_market_cap']
low_priority_cols = ['market_entropy', 'total_market_cap', 'market_cap']
all_numeric_cols = [] + numeric_cols
for column in high_priority_cols:
    for center_of_mass in [2, 4, 12, 24, 72]:
        new_col_name = column + '_MA_' + str(window)
        all_numeric_cols.append(new_col_name)
        ma = subset_full_data.groupby("refID_coin", as_index = False).apply(lambda x: x[column].ewm(com=center_of_mass).mean())
        subset_full_data[new_col_name] = ma.reset_index(level=0, drop=True)
        
for column in high_priority_cols:
    for center_of_mass in [2, 4]:
        new_col_name = column + '_MA_' + str(window)
        all_numeric_cols.append(new_col_name)
        ma = subset_full_data.groupby("refID_coin", as_index = False).apply(lambda x: x[column].ewm(com=center_of_mass).mean())
        subset_full_data[new_col_name] = ma.reset_index(level=0, drop=True)

### Add target:

In [ ]:
subset_full_data['target'] = subset_full_data.groupby(['refID_coin'])['price'].shift(-1)

### Add price change trend:

In [ ]:
for com in [1,2,4,6,12, 24, 48]:
    subset_full_data['trend_price_' + str(com)] = subset_full_data.groupby("refID_coin", as_index = False)\
        .apply(lambda x: (x.target - x.price).ewm(com=com).std())\
        .reset_index(level=0, drop=True)

### Write to file:

In [ ]:
subset_full_data.to_csv('data/cleaned.csv')

### Create sample test file:

In [ ]:
start_test_date = '2018-02-19'
crypto = 'Ripple'
subset = subset_full_data[(subset_full_data.refID_coin == 'Ripple') &
                          (subset_full_data.time >= start_test_date)]
subset = subset.iloc[0:3168][['time', 'refID_coin', 'price']]
subset.iloc[2880:, 2] = np.nan
subset.to_csv('data/test/test_01.csv')

start_test_date = '2018-03-01'
crypto = 'Ethereum'
subset = subset_full_data[(subset_full_data.refID_coin == 'Ripple') &
                          (subset_full_data.time >= start_test_date)]
subset = subset.iloc[0:3168][['time', 'refID_coin', 'price']]
subset.iloc[2880:, 2] = np.nan
subset.to_csv('data/test/test_02.csv')